In [1]:
import pandas as pd
import requests

In [2]:
#General stats table
response = requests.get('https://www.chess.com/callback/member/stats/juslane')
stats = pd.DataFrame(response.json()['stats'])
stats

,key,stats,gameCount,lastPlayed,lastDate
0,tactics,"{'rating': 2093, 'highest_rating': 2371, 'high...",12612,False,2024-07-07T12:27:55-07:00
1,chess,"{'rating': 1036, 'highest_rating': 1204, 'high...",12,True,2024-05-20T12:11:20-07:00
2,tactics_challenge,"{'highest_score': 34, 'highest_type_code': 'th...",109,False,NaN
3,chess960,"{'rating': 740, 'highest_rating': 800, 'highes...",1,False,2024-04-09T20:29:34-07:00
4,bullet,"{'rating': 854, 'highest_rating': 854, 'highes...",2,False,2023-12-14T21:03:50-08:00
5,lightning,"{'rating': 747, 'highest_rating': 753, 'highes...",43,False,2021-06-03T23:16:59-07:00


In [3]:
#Puzzle Data
puzzles = stats[stats['key'] == 'tactics']
puzzles

,key,stats,gameCount,lastPlayed,lastDate
0,tactics,"{'rating': 2093, 'highest_rating': 2371, 'high...",12612,False,2024-07-07T12:27:55-07:00


In [4]:
#Total puzzles played
puzzles_played = puzzles['gameCount'][0]
puzzles_played

12612

In [5]:
puzzle_ratings = pd.DataFrame(puzzles['stats'][0], index=['ratings']).reset_index(drop=True)
puzzle_ratings

,rating,highest_rating,highest_rating_date,rating_time_change_days,rating_time_change_value,highest_rating_info,lowest_rating_info,total_seconds,lowest_rating,lowest_rating_date,attempt_count,passed_count,failed_count,last_date
0,2093,2371,1695452747,30,6,NaN,NaN,600706,412,1615954266,12612,6517,6095,2024-07-07T12:27:55-07:00


In [6]:
# Get Player Puzzle Rating
player_rating = puzzle_ratings['rating'][0]
player_rating

2093

In [7]:
# Time Spent
hours = divmod(puzzle_ratings["total_seconds"][0], 3600)
minutes = divmod(hours[1], 60)[0]

print(f'{hours[0]} Hours and {minutes} Minutes')
print(f'Spent on {puzzles_played} Puzzles')

166 Hours and 51 Minutes
Spent on 12612 Puzzles


In [8]:
# Efficiency Rating

pass_perc = puzzle_ratings['passed_count'][0] / (puzzle_ratings['passed_count'][0] + puzzle_ratings['failed_count'][0])
print(f'{round(pass_perc*100, 2)}% of Puzzles Passed')

51.67% of Puzzles Passed


In [ ]:
# Find Global and Regional Rankings

page_nums_global = [i for i in range(2500, 5000)]
page_nums_regional = [i for i in range(700, 3000)]

running_0 = True
global_rank = 0
for page in page_nums_global:
    response = requests.get('https://www.chess.com/callback/leaderboard/tactics', params={'page':page})
    df = pd.DataFrame(response.json()['leaders'])
    for player in range(0, 50):
        try:
            global_rank = df[df['score'] == player_rating]['rank'][player]
            print(f'Global Rank: {global_rank}')
            running_0 = False
            break
        except:
            pass
    if not running_0:
        break
    else:
        print(page)
        
running = True
regional_rank =0
for page in page_nums_regional:
    response = requests.get('https://www.chess.com/callback/leaderboard/tactics', params={'page':page, 'country':'US'})
    df = pd.DataFrame(response.json()['leaders'])
    for player in range(0, 50):
        try:
            regional_rank = df[df['score'] == player_rating]['rank'][player]
            print(f'U.S. Rank: {regional_rank}')
            running = False
            break
        except:
            pass
    if not running:
        break

### Need to develop another strategy
One limitation is the number of GET requests being made. Per request, the max rating per page should influence the next page to search. For example, if the first page searched has a max rating of 4000 and your player rating is 1600, the algorithm should not search for the next page, but skip a few and try again. The amount skipped should be influenced by the gap between the max score of the current page and the player score. 

In [ ]:
def parse_page(parse_type, search_page):
    '''Return min and max score on current search page as well as Pandas dataframe of page scores.
        parse_type can be set to "regional", otherwise global leaderboard is searched.'''
   
    if parse_type != 'regional':
        response = requests.get('https://www.chess.com/callback/leaderboard/tactics', 
                                params={'page':search_page})
        df = pd.DataFrame(response.json()['leaders'])
        max_page_score = df['score'].max()
        min_page_score = df['score'].min()
        return df, max_page_score, min_page_score
    else:
        response = requests.get('https://www.chess.com/callback/leaderboard/tactics', 
                                params={'page':search_page, 'country':'US'})
        df = pd.DataFrame(response.json()['leaders'])
        max_page_score = df['score'].max()
        min_page_score = df['score'].min()
        return df, max_page_score, min_page_score
    
def turn_pages(current_value, printing=None):
    '''Stages the correct page values for the next search.
        current_value should be set according to whether target value above, below, or equal to comparison value.
        printing can be set to True to print the current data.'''
    
    global search_page, last_page_searched, second_last_page_searched, last_page_above_rating, last_page_below_rating
    
    if current_value == 'below':
        if printing:
            print(f'\n\nCurrent Search Page: {search_page}\nLast Search Page:{last_page_searched}\n2nd Last Search Page: {second_last_page_searched}\nLast Page Above Rating: {last_page_above_rating}')
        second_last_page_searched = last_page_searched
        last_page_searched = search_page
        last_page_above_rating = search_page
        search_page = int((search_page + last_page_below_rating) / 2)
        if printing:
            print(f'\n\nCurrent Search Page: {search_page}\nLast Search Page:{last_page_searched}\n2nd Last Search Page: {second_last_page_searched}\nLast Page Above Rating: {last_page_above_rating}')
        
    elif current_value == 'above':
        if printing:
            print(f'\n\nCurrent Search Page: {search_page}\nLast Search Page:{last_page_searched}\n2nd Last Search Page: {second_last_page_searched}\nLast Page Above Rating: {last_page_below_rating}')
        second_last_page_searched = last_page_searched
        last_page_searched = search_page
        last_page_below_rating = search_page
        search_page = int((search_page + last_page_above_rating) / 2)
        if printing:
            print(f'\n\nCurrent Search Page: {search_page}\nLast Search Page:{last_page_searched}\n2nd Last Search Page: {second_last_page_searched}\nLast Page Above Rating: {last_page_below_rating}')
    
    elif current_value == 'equal':
        if printing:
            print(f'\n\nCurrent Search Page: {search_page}\nLast Search Page:{last_page_searched}\n2nd Last Search Page: {second_last_page_searched}')
        second_last_page_searched = last_page_searched
        last_page_searched = search_page
        last_page_below_rating = search_page
        search_page -= 1
        if printing:
            print(f'\n\nCurrent Search Page: {search_page}\nLast Search Page:{last_page_searched}\n2nd Last Search Page: {second_last_page_searched}')
    
    else:
        print('\nSomething went wrong with turn_pages function...\n')
        print(f'\n\nCurrent Search Page: {search_page}\nLast Search Page:{last_page_searched}\n2nd Last Search Page: {second_last_page_searched}\nLast Page Above Rating: {last_page_above_rating}')

In [14]:
search_page = 1
highest_page_possible = 10000
second_last_page_searched = highest_page_possible
last_page_searched = 0
last_page_below_rating = 0
last_page_above_rating = 0
running = True
regional_rank = 0
global_rank = 0
run_count = 0
parse_type = 'regional'
total_global_players = 3744704

print('Searching...')
while running:
    df, max_page_score, min_page_score = parse_page(parse_type=parse_type, search_page=search_page)
    
    if player_rating >= min_page_score and player_rating <= max_page_score:
        try:
            print(f'Player Rating was ON THE MONEY; {search_page} + {last_page_searched}) / 2 = {(search_page + last_page_searched) / 2}\nVariables:\n\nsearch_page: {search_page}\nhighest_page_possible: {highest_page_possible}\nlast_page_searched: {last_page_searched}\nsecond_last_page_searched: {second_last_page_searched}\nlast_page_below_rating: {last_page_below_rating}\nlast_page_above_rating: {last_page_above_rating}\nrunning: {running}\nglobal_rank: {global_rank}\nrun_count: {run_count}\nPlayer Rating: {player_rating}\nmin_page_score: {min_page_score}\nmax_page_score: {max_page_score}\n\n\n')
            if parse_type == 'regional':
                regional_rank = df[df['score'] == player_rating]['rank'][player]
                running = False
                
            else:
                global_rank = df[df['score'] == player_rating]['rank'][player]
                search_page = 1
                second_last_page_searched = highest_page_possible
                last_page_searched = 0
                last_page_below_rating = 0
                last_page_above_rating = 0
                run_count = -1
                parse_type = 'regional'

        except:
            print(f'EXCEPT\nPlayer Rating was Equal to Min. Page Score; {search_page} + {last_page_searched}) / 2 = {(search_page + last_page_searched) / 2}\nVariables:\n\nsearch_page: {search_page}\nhighest_page_possible: {highest_page_possible}\nlast_page_searched: {last_page_searched}\nsecond_last_page_searched: {second_last_page_searched}\nlast_page_below_rating: {last_page_below_rating}\nlast_page_above_rating: {last_page_above_rating}\nrunning: {running}\nglobal_rank: {global_rank}\nrun_count: {run_count}\nPlayer Rating: {player_rating}\nmin_page_score: {min_page_score}\nmax_page_score: {max_page_score}\n\n\n')
            df, max_page_score, min_page_score = parse_page(parse_type=parse_type, search_page=last_page_searched)
            if parse_type == 'regional':
                regional_rank = df['rank'][0]
                running = False
                
            else:
                global_rank = df['rank'][0]
                search_page = 1
                second_last_page_searched = highest_page_possible
                last_page_searched = 0
                last_page_below_rating = 0
                last_page_above_rating = 0
                run_count = -1
                parse_type = 'regional'
                    
            pass
    
    elif player_rating < min_page_score:
        print(f'Player Rating was LOWER than Min. Page Score; {search_page} + {highest_page_possible}) / 2 = {(search_page + highest_page_possible) / 2}\nVariables:\n\nsearch_page: {search_page}\nhighest_page_possible: {highest_page_possible}\nlast_page_searched: {last_page_searched}\nsecond_last_page_searched: {second_last_page_searched}\nlast_page_below_rating: {last_page_below_rating}\nlast_page_above_rating: {last_page_above_rating}\nrunning: {running}\nglobal_rank: {global_rank}\nrun_count: {run_count}\nPlayer Rating: {player_rating}\nmin_page_score: {min_page_score}\nmax_page_score: {max_page_score}\n\n\n')
        if run_count == 0:
            last_page_searched = search_page
            search_page = highest_page_possible
        else:
            turn_pages('below')
            
    elif player_rating > max_page_score:
        print(f'Player Rating was HIGHER than Min. Page Score; {search_page} / 2 = {search_page/2}\nVariables:\n\nsearch_page: {search_page}\nhighest_page_possible: {highest_page_possible}\nlast_page_searched: {last_page_searched}\nsecond_last_page_searched: {second_last_page_searched}\nrunning: {running}\nglobal_rank: {global_rank}\nrun_count: {run_count}\nPlayer Rating: {player_rating}\nmin_page_score: {min_page_score}\nmax_page_score: {max_page_score}\n\n\n')
        turn_pages('above')
        
    else:
        print('why')
    if not running:
        print(f'U.S. Rank: {regional_rank}')
        print('Global Rank: {} ({:0.2f}th Percentile)'.format(global_rank, global_rank/total_global_players*100.00))
        print(f'Global Player Count: {total_global_players}')
        break
        
    run_count += 1
    print(f'Run Count: {run_count}\nSearch Page: {search_page}\nLast Page: {last_page_searched}\n\n')
    

Searching...
Player Rating was LOWER than Min. Page Score; 1 + 10000) / 2 = 5000.5
Variables:

search_page: 1
highest_page_possible: 10000
last_page_searched: 0
second_last_page_searched: 10000
last_page_below_rating: 0
last_page_above_rating: 0
running: True
global_rank: 0
run_count: 0
Player Rating: 2093
min_page_score: 4682
max_page_score: 47595



Run Count: 1
Search Page: 10000
Last Page: 1


Player Rating was HIGHER than Min. Page Score; 10000 / 2 = 5000.0
Variables:

search_page: 10000
highest_page_possible: 10000
last_page_searched: 1
second_last_page_searched: 10000
running: True
global_rank: 0
run_count: 1
Player Rating: 2093
min_page_score: 1666
max_page_score: 1666



Run Count: 2
Search Page: 5000
Last Page: 10000


Player Rating was HIGHER than Min. Page Score; 5000 / 2 = 2500.0
Variables:

search_page: 5000
highest_page_possible: 10000
last_page_searched: 10000
second_last_page_searched: 1
running: True
global_rank: 0
run_count: 2
Player Rating: 2093
min_page_score: 1987

### Last thing to solve for is total puzzle player count

In [ ]:
You also have to be good enough (probably better than 2100) to even appear on these boards. A 2060 rating wasn't appearing. Get gud.